In [1]:
from simulate_2 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

/opt/miniconda3/envs/ML/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:

path_eval_data = 'data/batch100'
df_videos = pd.read_csv('data/videos_all_with_result.csv')
df_videos.fillna(0, inplace=True)

df_list = []

for file in os.listdir(path_eval_data):
    
    if file.endswith('.csv'):
        file_path = os.path.join(path_eval_data, file)
        df = pd.read_csv(file_path)
        
        df_list.append(df)

all_data = pd.concat(df_list, ignore_index=True)

models = ['gen2', 'latte', 'pika', 'vgen', 'videocrafter2']
# Bootstrap
G = 100

bs_results = []

for _ in range(G):
    
    bs_df_list = []
    
    #resampling
    for df in df_list:
        indices = np.random.choice(df.index, len(df))
        bs_df_list.append(df.loc[indices])
        
    result = simulate_all(bs_df_list, df_videos)
    
    bs_results.append(result)
    
    print('Bootstrap {} done'.format(_+1))

result_df_list = [pd.DataFrame(columns=models) for _ in range(6)]

for result in bs_results:
    for key in result.keys():
        result_df_list[key-1] = pd.concat([result_df_list[key-1], 
                                           pd.DataFrame([result[key]])], 
                                          ignore_index=True, axis=0)

# mkdir data/bs_result if not exist
if not os.path.exists('data/bs_result'):
    os.makedirs('data/bs_result')

for i in range(6):
    result_df_list[i].to_csv('data/bs_result/bs_result_{}.csv'.format(i+1), index=False)
    

In [2]:
from get_rate import fit_models

path_eval_data = 'data/batch100'
df_videos = pd.read_csv('data/videos_all_with_result.csv')
df_videos.fillna(0, inplace=True)

df_list = []

for file in os.listdir(path_eval_data):
    
    if file.endswith('.csv'):
        file_path = os.path.join(path_eval_data, file)
        df = pd.read_csv(file_path)
        
        df_list.append(df)

all_data = pd.concat(df_list, ignore_index=True)
models = ['gen2', 'latte', 'pika', 'vgen', 'videocrafter2']
all_data.head()

,pairs_id,model_1,model_2,video_url_1,video_url_2,dimension,rating
0,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,1,1
1,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,2,1
2,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,3,1
3,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,4,0
4,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,5,0


In [3]:
videos = pd.read_csv('data/videos_all_with_result.csv')
videos.head()

,id,models,video_url,prompt,image_url,subject_consistency,temporal_flickering,motion_smoothness,dynamic_degree,aesthetic_quality,imaging_quality,overall_consistency
0,1,gen2,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant,NaN,93.273244,0.996796,0.996651,0,0.571255,36.246704,0.288979
1,2,gen2,https://video-generation-result2.s3.ap-northea...,bird nests on a tree captured with moving camera,NaN,94.544629,0.992363,0.995272,0,0.632408,77.747590,0.259322
2,3,gen2,https://video-generation-result2.s3.ap-northea...,close up video of snail,NaN,94.229310,0.997329,0.996740,0,0.637062,60.462318,0.209860
3,4,gen2,https://video-generation-result2.s3.ap-northea...,ants eating dead spider,NaN,89.503513,0.987959,0.994335,0,0.422831,61.973820,0.246135
4,5,gen2,https://video-generation-result2.s3.ap-northea...,a frog eating an ant,NaN,94.105455,0.992855,0.995771,0,0.693387,56.432156,0.226263


In [4]:
all_data_merged = pd.merge(all_data, videos[['video_url', 'prompt']], left_on='video_url_1', right_on='video_url', how='left')
all_data_merged.isna().sum()

pairs_id       0
model_1        0
model_2        0
video_url_1    0
video_url_2    0
dimension      0
rating         0
video_url      0
prompt         0
dtype: int64

In [ ]:
# # Constants
# G = 100
# dimension_names = {
#     1: "Video Quality",
#     2: "Temporal Quality",
#     3: "Motion Quality",
#     4: "Text Alignment",
#     5: "Ethical Robustness",
#     6: "Human Preference"
# }
# column_names = ['Latte', 'Pika', 'TF-T2V', 'Gen2', 'Videocrafter2']

# # Prepare DataFrame
# bs_result_with_n = pd.DataFrame()

# # Sample sizes
# nums = list(range(10, 210, 20))

# # Unique prompts
# all_prompts = all_data_merged['prompt'].unique()

# for n in nums:
#     prompts = np.random.choice(all_prompts, n, replace=False)
#     sub_data = all_data_merged[all_data_merged['prompt'].isin(prompts)]
    
#     # Collect all bootstrap results first
#     bs_results = [fit_models(sub_data.sample(frac=1, replace=True)) for _ in range(G)]
    
#     print(f'Bootstrapping for n={n} done.')

#     # Initialize a list of empty DataFrames for each dimension
#     result_df_list = [pd.DataFrame(columns=column_names) for _ in range(6)]
    
#     # Accumulate results
#     for result in bs_results:
#         for key, value in result.items():
#             result_df_list[key-1] = pd.concat([result_df_list[key-1], pd.DataFrame([value])], ignore_index=True)
    
#     # Format and append results
#     for i, df in enumerate(result_df_list):
#         melted_df = df.melt(var_name='model', value_name='estimation')
#         melted_df['dimension'] = dimension_names[i+1]
#         melted_df['n'] = n
#         bs_result_with_n = pd.concat([bs_result_with_n, melted_df], ignore_index=True)

# # Save to CSV
# bs_result_with_n.to_csv('data/bs_result/bs_result_with_n.csv', index=False)

In [6]:
all_data_merged

,pairs_id,model_1,model_2,video_url_1,video_url_2,dimension,rating,video_url,prompt
0,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,1,1,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant
1,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,2,1,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant
2,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,3,1,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant
3,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,4,0,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant
4,1281,gen2,latte,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,5,0,https://video-generation-result2.s3.ap-northea...,macro shot of ladybug on green leaf plant
...,...,...,...,...,...,...,...,...,...
59995,1630,vgen,videocrafter2,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,2,1,https://video-generation-result2.s3.ap-northea...,slow motion footage of a racing car
59996,1630,vgen,videocrafter2,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,3,1,https://video-generation-result2.s3.ap-northea...,slow motion footage of a racing car
59997,1630,vgen,videocrafter2,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,4,0,https://video-generation-result2.s3.ap-northea...,slow motion footage of a racing car
59998,1630,vgen,videocrafter2,https://video-generation-result2.s3.ap-northea...,https://video-generation-result2.s3.ap-northea...,5,0,https://video-generation-result2.s3.ap-northea...,slow motion footage of a racing car


In [ ]:
import concurrent.futures
from simulate_2 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from get_rate import fit_models

# Function to perform bootstrap and fit models
def bootstrap_and_fit(sub_data):
    sub_data = sub_data.sample(frac=1, replace=True)
    result = dict()
    for dim in range(1, 7):
        result[dim] = fit_models(sub_data[sub_data['dimension'] == dim])
    return result


path_eval_data = 'data/batch100'
df_videos = pd.read_csv('data/videos_all_with_result.csv')
df_videos.fillna(0, inplace=True)

df_list = []

for file in os.listdir(path_eval_data):
    
    if file.endswith('.csv'):
        file_path = os.path.join(path_eval_data, file)
        df = pd.read_csv(file_path)
        
        df_list.append(df)

all_data = pd.concat(df_list, ignore_index=True)
models = ['gen2', 'latte', 'pika', 'vgen', 'videocrafter2']
all_data.head()
videos = pd.read_csv('data/videos_all_with_result.csv')
all_data_merged = pd.merge(all_data, videos[['video_url', 'prompt']], 
                           left_on='video_url_1', right_on='video_url', 
                           how='left')

# Constants
G = 100
dimension_names = {
    1: "Video Quality",
    2: "Temporal Quality",
    3: "Motion Quality",
    4: "Text Alignment",
    5: "Ethical Robustness",
    6: "Human Preference"
}
column_names = ['Latte', 'Pika', 'TF-T2V', 'Gen2', 'Videocrafter2']

# Initialize the DataFrame to collect results
bs_result_with_n = pd.DataFrame()

# Sample sizes
nums = list(range(10, 210, 20))

# Unique prompts
all_prompts = all_data_merged['prompt'].unique()

if __name__ == '__main__':

    for n in nums:
        prompts = np.random.choice(all_prompts, n, replace=False)
        sub_data = all_data_merged[all_data_merged['prompt'].isin(prompts)]
        
        # Parallel processing
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(executor.map(bootstrap_and_fit, [sub_data] * G))
        
        print(f'Bootstrapping for n={n} completed.')

        # Initialize a list of empty DataFrames for each dimension
        result_df_list = [pd.DataFrame(columns=column_names) for _ in range(6)]
        
        # Accumulate results
        for result in results:
            for key, value in result.items():
                result_df_list[key-1] = pd.concat([result_df_list[key-1], pd.DataFrame([value])], ignore_index=True)
        
        # Format and append results
        for i, df in enumerate(result_df_list):
            melted_df = df.melt(var_name='model', value_name='estimation')
            melted_df['dimension'] = dimension_names[i+1]
            melted_df['n'] = n
            bs_result_with_n = pd.concat([bs_result_with_n, melted_df], ignore_index=True)

    # Save to CSV
    bs_result_with_n.to_csv('data/bs_result/bs_result_with_n.csv', index=False)
# bs_result_with_n.head()


In [ ]:
# import concurrent.futures
# from simulate_2 import *
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# from get_rate import fit_models

# # Constants
# G = 100
# dimension_names = {
#     1: "Video Quality",
#     2: "Temporal Quality",
#     3: "Motion Quality",
#     4: "Text Alignment",
#     5: "Ethical Robustness",
#     6: "Human Preference"
# }
# column_names = ['Latte', 'Pika', 'TF-T2V', 'Gen2', 'Videocrafter2']

# # Initialize the DataFrame to collect results
# bs_result_with_n = pd.DataFrame()

# # Sample sizes
# nums = list(range(10, 210, 20))

# # Unique prompts
# all_prompts = all_data_merged['prompt'].unique()

# # Function to perform bootstrap and fit models
# def bootstrap_and_fit(sub_data):
#     return fit_models(sub_data.sample(frac=1, replace=True))

# if __name__ == '__main__':
#     for n in nums:
#         prompts = np.random.choice(all_prompts, n, replace=False)
#         sub_data = all_data_merged[all_data_merged['prompt'].isin(prompts)]
        
#         # Parallel processing
#         with concurrent.futures.ProcessPoolExecutor() as executor:
#             results = list(executor.map(bootstrap_and_fit, [sub_data] * G))
        
#         print(f'Bootstrapping for n={n} completed.')

#         # Initialize a list of empty DataFrames for each dimension
#         result_df_list = [pd.DataFrame(columns=column_names) for _ in range(6)]
        
#         # Accumulate results
#         for result in results:
#             for key, value in result.items():
#                 result_df_list[key-1] = pd.concat([result_df_list[key-1], pd.DataFrame([value])], ignore_index=True)
        
#         # Format and append results
#         for i, df in enumerate(result_df_list):
#             melted_df = df.melt(var_name='model', value_name='estimation')
#             melted_df['dimension'] = dimension_names[i+1]
#             melted_df['n'] = n
#             bs_result_with_n = pd.concat([bs_result_with_n, melted_df], ignore_index=True)
    
#     bs_result_with_n.to_csv('data/bs_result/bs_result_with_n.csv', index=False)
# bs_result_with_n.head()


In [46]:
results[1]

(None,
 {'vgen': 0.584750017380269,
  'latte': 0.8554539714819328,
  'pika': 1.1565286634719643,
  'gen2': 2.905565129593506,
  'videocrafter2': 0.48214668085587536})